In [ ]:
import pandas as pd
import numpy as np
from pysus.online_data import SINAN, FTP_Inspect, parquets_to_dataframe
#from pysus.preprocessing.decoders import decodifica_idade_SINAN
#decodifica_idade_SINAN

# Regiões, Idade, Ano, '

In [ ]:
years = SINAN.get_available_years('Poliomielite')
print(years)

In [ ]:
df = pd.DataFrame()
for y in years:
    fn = SINAN.download('Poliomielite', y)
    df2 = parquets_to_dataframe(fn)
    df = pd.concat([df, df2])

In [ ]:
df.shape

In [ ]:
for cname in df.columns:
    if cname.startswith('DT_'):
        df[cname] = pd.to_datetime(df[cname], errors='coerce')
    elif cname.startswith('ID_'):
        try:
            df[cname] = pd.to_numeric(df[cname])
        except ValueError:
            continue

In [ ]:
df['SG_UF_NOT'] = df.SG_UF_NOT.astype(int)
df[df.SG_UF_NOT==31].to_csv('Polio_mod.csv',sep=';',compression='zip')

In [ ]:
display(df)

In [ ]:
def func(a):
    estados = {
    '11': 'RO', '12': 'AC', '13': 'AM', '14': 'RR', '15': 'PA',
    '16': 'AP', '17': 'TO', '21': 'MA', '22': 'PI', '23': 'CE',
    '24': 'RN', '25': 'PB', '26': 'PE', '27': 'AL', '28': 'SE',
    '29': 'BA', '31': 'MG', '32': 'ES', '33': 'RJ', '35': 'SP',
    '41': 'PR', '42': 'SC', '43': 'RS', '50': 'MS', '51': 'MT',
    '52': 'GO', '53': 'DF'
}
    return estados[a]
    
df['SG_UF'] = df['SG_UF'].apply(lambda x: func(x))

In [ ]:
print(df['SG_UF'].isnull().sum())
print(df['DT_NOTIFIC'].isnull().sum())


In [36]:
df['SG_UF']

0      SP
1      AM
2      GO
3      AM
4      SC
5      MG
6      PE
7      AC
8      RN
9      RO
10     RJ
11     RJ
12     RS
13     MT
14     RS
15     RJ
16     PE
17     MG
18     BA
19     PB
20     RS
21     MA
22     BA
23     AC
24     SP
25     GO
26     BA
27     MG
28     SE
29     BA
30     GO
31     MG
32     SP
33     SP
34     BA
35     MS
36     MS
37     CE
38     SP
39     BA
40     PE
41     ES
42     CE
43     BA
44     ES
45     MA
46     AL
47     MG
48     RO
49     SP
50     PE
51     DF
52     MG
53     GO
54     RS
55     GO
56     DF
57     DF
58     BA
59     MS
60     PA
61     BA
62     MS
63     SP
64     RS
65     SC
66     SP
67     PI
68     PE
69     RO
70     PE
71     RO
72     SP
73     GO
74     PI
75     PI
76     GO
77     GO
78     BA
79     AL
80     MS
81     AM
82     MG
83     MG
84     RO
85     PE
86     RN
87     GO
88     PI
89     AM
90     PA
91     PI
92     PB
93     SE
94     SC
95     DF
96     RS
97     PE
98     PB
99     PB


In [37]:
df['DT_NOTIFIC']

0     2012-01-05
1     2012-01-06
2     2012-01-09
3     2012-01-09
4     2012-01-09
5     2012-01-10
6     2012-01-10
7     2012-01-10
8     2012-01-10
9     2012-01-11
10    2012-01-11
11    2012-01-11
12    2012-01-13
13    2012-01-13
14    2012-01-16
15    2012-01-16
16    2012-01-16
17    2012-01-17
18    2012-01-18
19    2012-01-19
20    2012-01-19
21    2012-01-20
22    2012-01-20
23    2012-01-20
24    2012-01-20
25    2012-01-23
26    2012-01-23
27    2012-01-23
28    2012-01-25
29    2012-01-26
30    2012-01-27
31    2012-01-27
32    2012-01-27
33    2012-01-30
34    2012-01-31
35    2012-02-01
36    2012-02-01
37    2012-02-02
38    2012-02-03
39    2012-02-03
40    2012-02-03
41    2012-02-03
42    2012-02-03
43    2012-02-03
44    2012-02-06
45    2012-02-07
46    2012-02-08
47    2012-02-08
48    2012-02-09
49    2012-02-09
50    2012-02-09
51    2012-02-10
52    2012-02-14
53    2012-02-14
54    2012-02-15
55    2012-02-15
56    2012-02-16
57    2012-02-16
58    2012-02-

In [ ]:
df = df[['SG_UF']]

In [39]:
import pandas as pd
df = pd.read_excel(
    'https://github.com/datagy/mediumdata/raw/master/sample_pivot.xlsx')

grouped = df.groupby(['Region', 'Type']).aggregate(
    {'Units': 'mean', 'Sales': ['min', 'max']})
print(grouped.head())

                                Units Sales      
                                 mean   min   max
Region Type                                      
East   Children's Clothing  20.513274    36  1122
       Men's Clothing       19.836066    36  1054
       Women's Clothing     19.159091    52  1054
North  Children's Clothing  20.741176    55  1088
       Men's Clothing             NaN    66  1155
